In [1]:
from rag import Custom_Rag


In [ ]:

F=Custom_Rag(model="gpt-4o")
F.main("Who is shyam?")

In [ ]:
l=Custom_Rag(model="meta/llama3-70b-instruct")
l.main("Who is shyam?")

In [ ]:
ll=Custom_Rag(model="gemma-7b-it")
ll.main("Who is shyam?")

In [ ]:
k=Custom_Rag(model="mixtral-8x7b-32768")
k.main("Who is shyam?")

## EVALUATION METRICS - LLM JUDGE

![image](rag_eval_overview-c0c3c2ac03f44b2b67c58ab67f32d82a.png)

In [2]:
from langchain import hub
from langchain_openai import ChatOpenAI

# Grade prompt
grade_prompt_answer_accuracy = prompt = hub.pull("langchain-ai/rag-answer-vs-reference")

def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs.get("prompt")
    reference = example.outputs.get("answer")
    prediction = run.outputs.get("response")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_answer_accuracy | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_v_reference_score", "score": score}

# Grade prompt
grade_prompt_answer_helpfulness = prompt = hub.pull("langchain-ai/rag-answer-helpfulness")

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs.get("prompt")
    prediction = run.outputs.get("response")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}

# Prompt
grade_prompt_hallucinations = prompt = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """

    # RAG inputs
    input_question = example.inputs.get("prompt")
    contexts = run.outputs.get("context")

    # RAG answer
    prediction = run.outputs.get("response")
    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

# Grade prompt
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """

    # RAG inputs
    input_question = example.inputs.get("prompt")
    contexts = run.outputs.get("context")

    # LLM grader
    llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [3]:
from langsmith.evaluation import  evaluate
from langsmith import Client

client = Client()
dataset_name = "raft-custom-dataset"
experiment_results = evaluate(
    lambda inputs: Custom_Rag(model="gpt-4o").main(inputs["prompt"]),
    data=dataset_name,
    evaluators=[answer_evaluator, answer_helpfulness_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="gpt-4o rag eval",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "LCEL context, gpt-4-turbo",
    }
)

c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o rag eval-7f707e27' at:
https://smith.langchain.com/o/b47753b3-6d84-5fa3-b240-64adbbe1c7be/datasets/f7abf090-86e8-4960-94f8-0fb9c3569c53/compare?selectedSessions=add979f1-1dc8-4b8d-9b29-1b77c64ce5ef




0it [00:00, ?it/s]

client=<openai.resources.chat.completions.Completions object at 0x000002002B32C2F0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002002B345340> model_name='gpt-4o' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x000002002B32DE80> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002002B5AD850> model_name='gpt-4o' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x000002002B32F800> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002002B5AE870> model_name='gpt-4o' temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''


c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
3it [01:07, 22.51s/it]


In [4]:
list_of_models=["gpt-3.5-turbo", "gpt-4o","gemma-7b-it", "mixtral-8x7b-32768", "llama3-8b-8192","databricks/dbrx-instruct", "microsoft/phi-3-small-8k-instruct", "google/gemma-7b", "meta/llama3-70b-instruct"]


In [5]:
for model in list_of_models:
    xperiment_results = evaluate(
    lambda inputs: Custom_Rag(model=model).main(inputs["prompt"]),
    data=dataset_name,
    evaluators=[answer_evaluator, answer_helpfulness_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix=f"{model} rag eval",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "LCEL context, gpt-4-turbo",
    },
    num_repetitions=3
)

View the evaluation results for experiment: 'gpt-3.5-turbo rag eval-1e65d1b8' at:
https://smith.langchain.com/o/b47753b3-6d84-5fa3-b240-64adbbe1c7be/datasets/f7abf090-86e8-4960-94f8-0fb9c3569c53/compare?selectedSessions=c4b059ed-af4b-47bc-9e43-61f0241e7014




0it [00:00, ?it/s]

client=<openai.resources.chat.completions.Completions object at 0x00000200016E23F0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000200016F9E20> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''client=<openai.resources.chat.completions.Completions object at 0x00000200083E76E0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000200016FAC90> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''
client=<openai.resources.chat.completions.Completions object at 0x00000200083E6E70> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000200016FBBF0> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''

client=<openai.resources.chat.completions.Completions object at 0x00000200016E37A0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020001707050> temperature=0.2 openai_api_key=SecretStr('**********') openai_proxy=''


c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
9it [01:17,  8.63s/it]


View the evaluation results for experiment: 'gpt-4o rag eval-1e6b6d4c' at:
https://smith.langchain.com/o/b47753b3-6d84-5fa3-b240-64adbbe1c7be/datasets/f7abf090-86e8-4960-94f8-0fb9c3569c53/compare?selectedSessions=891de551-c8a6-4bd3-b4b3-77c88edba0e2




...

c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
9it [01:31, 10.17s/it]


View the evaluation results for experiment: 'meta/llama3-70b-instruct rag eval-ec45d1f6' at:
https://smith.langchain.com/o/b47753b3-6d84-5fa3-b240-64adbbe1c7be/datasets/f7abf090-86e8-4960-94f8-0fb9c3569c53/compare?selectedSessions=1fc5596d-35af-48f0-9df6-e48c787ee512




0it [00:00, ?it/s]

model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2
model='meta/llama3-70b-instruct' temperature=0.2


c:\Users\shyams\Downloads\projects\rag with langsmith evaluation\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
9it [01:26,  9.64s/it]
